# Foundation 6. 클로저 (학습)

## 6.1.변수(variable)와 범위(scope)  

### 용어

함수 밖에서 정의된 변수를 **전역 변수**,  

함수 안에서 정의된 변수를 **지역 변수**,  

각 변수가 적용되는 영역을 **범위**라고 한다.  

### 특성

각 범위에 따른 변수의 기본적인 특징이다.

전역 변수는 `함수 밖에서만 수정`이 가능하다.  

지역 변수는 `함수 안에서만 적용`되고 `수정`되어, 함수 밖에서는 사용할 수 없다.  

### 각 변수에 대한 수정 방법  

함수 안에서 전역 변수 정의·수정하기: 함수 안에서 `global` 명령어를 사용

지역 변수 범위를 전역으로 설정: 함수 안에서 `nonlocal` 명령어 사용  

지역 변수의 범위

In [4]:
def hello():
    who2 = '인유'  # 지역 변수 생성
    def hello2():
        nonlocal who2  # 주석을 씌우고 돌려봅시다!
        who2 = '아이펠'

    hello2()  # 안에 있던 hello2() 함수 실행
    print(f'hello {who2}!')


hello()

"""hello2()에서 nonlocal로 who2를 함수 밖에서도 사용하도록 조정했기 때문에 who2에 '아이펠'이 할당되었다."""


hello 아이펠!


"hello2()에서 nonlocal로 who2를 함수 밖에서도 사용하도록 조정했기 때문에 who2에 '아이펠'이 할당되었다."

### 전역 변수 - nonlocal의 우선순위  

가까운 함수부터 지역 변수를 찾는다. 할당되지 않은 변수에 대해서는 그 바깥 가장 가까운 변수를 불러온다.

In [9]:
def A(): 
    x = 10
    y = 20
    def B():
        x = 30
        def C():
            nonlocal x  # 10을 바인딩할지, 30을 바인딩 할지
            nonlocal y  # 함수 하나를 건너 뛰어서 y와 바인딩이 가능할지
            print(f"x(before) = {x}"); print(f"y(before) = {y}")
            x = x + 40
            y = y + 50
            print(f"\nx(after) = {x}"); print(f"y(after) = {y}")

        C()

    B()

A()


"""x는 가장 가까운 B()에서 정의된 30이, y는 그 다음 A()에서 정의된 20이 적용되었다."""


x(before) = 30
y(before) = 20

x(after) = 70
y(after) = 70


'x는 가장 가까운 B()에서 정의된 30이, y는 그 다음 A()에서 정의된 20이 적용되었다.'

## 6.2. 클로저(Closure)

어떤 함수의 **내부 함수**가 외부 함수의 변수(*프리변수)를 참조할 때,  

외부 함수가 **종료된 후**에도 내부 함수가 **외부 함수의 변수를 참조**할 수 있도록 **어딘가에 저장**하는 함수

`프리 변수`: 실제로 사용되지만, 내부에 선언되지는 않은 변수

아래 세 조건을 만족해야 한다.  
1. 어떤 함수의 내부 함수일 것  
2. 해당 내부함수가 외부 함수의 변수를 참조할 것  
3. 외부 함수가 내부 함수를 `return`할 것

In [10]:
# 클로저 함수의 예시 - my()가 클로저 함수
def intro(home):
    introduction = 'My hometown is '+home+'.'

    def my():  # 조건 1 충족 - my()는 intro()의 내부함수이다
        print(introduction)  # 조건 2 충족 - 외부함수 intro()의 변수, introduction을 참조한다
    
    return my  # 조건 3 충족 - 외부함수 intro()가 내부 함수 my를 return한다

### 특징 

자신을 둘러싼 스코프(*네임스페이스)의 상태 값을 기억한다  
- `스코프(네임스페이스)`: 내부 식별자를 구분하는 범위  

클로저는 길이가 1인 튜플이다.  
클로저가 기억하는 스코프의 상태는 `함수명.__closure__[0].cell_contents`에 담겨있다

#### 장점
1. 외부 함수가 삭제되어도 데이터가 클로저에 연결되기 때문에 장점 활용 가능
2. 전역 변수 남용을 방지, 변수가 많은 상황에서 섞이는 상황을 방지할 수 있다

#### 단점 
1. 스코프 유지를 위한 메모리 사용량이 상대적으로 많은 편이다

In [21]:
def test_func(input='abc'):
    print(input)
    return 'Executed Function'


print(test_func) # 정의된 함수명만 출력하면 함수 객체가 출력
print(test_func()) # ()로 실행하면 함수 내용이 출력. return 값이 없으면 None 출력


<function test_func at 0x000002A25AFDEA70>
abc
Executed Function


In [11]:
def intro(home):
# --------------- my() 함수의 클로저 영역 ---------------- # 

    introduction = 'My hometown is '+home+'.'

    def my():  
        print(introduction)  

# --------------- my() 함수의 클로저 영역 ---------------- #     
    return my  


f = intro('Yeosu')  # 1 'Yeosu'를 인수로 전달한 intro 함수
del intro  # 2 intro 함수 자체를 제거하더라도
f()  # 3 내부함수의 기능이 실행된다


My hometown is Yeosu.


In [13]:
f

'''intro() 함수가 my() 함수를 return하므로 f의 본체는 my()'''


<function __main__.intro.<locals>.my()>

In [15]:
intro

'''f를 실행하면 my() 기능이 동작하지만, intro 함수 자체는 사라졌다'''


NameError: name 'intro' is not defined

In [27]:
dir(f)[:5] # f 함수의 4번째에 closure이 있다.

# closure의 타입 = 튜플
type(f.__closure__)


tuple

In [32]:
# closure 객체의 길이 = 1
len(f.__closure__)


1

In [41]:
# closure 객체 내부 확인
dir(f.__closure__[0])


['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'cell_contents']

In [47]:
# closure 객체 내부 확인
f.__closure__[0].cell_contents


'My hometown is Yeosu.'